## Retriveing Linz traffic data


Before we start the exploration of the traffic data and test different traffic forecast model, we need to retrive the data and put in a form of time series using pandas library and then we can just dump the result on cvs file for a finer analysis.

In [1]:
import urllib, json, objectpath
from pprint import pprint

In [2]:
def get_data(route):
    response = urllib.urlopen('http://pendlr.risc-software.at/log?id=["%s"]&num=100000' % route)
    data = json.loads(response.read())
    tree_obj = objectpath.Tree(data)
    return tuple(tree_obj.execute('$..entries'))
def get_route():
    response = urllib.urlopen('http://pendlr.risc-software.at/log?num=100000')
    data = json.loads(response.read())
    tree_obj = objectpath.Tree(data)
    return tuple(tree_obj.execute('$..id'))

The methods above will return json files with the relevent inforamtion, the routes names and the corresponding json. After that we will translate the data into a pandas dataframe.

In [3]:
routes = get_route()
cur_data   = {}

for r in routes :
    if 'cur.' in r:
        print "-- route : ", r
        cur_data[r] = get_data(r)

-- route :  cur.traun.b1.traffic.risc-software.at
-- route :  cur.zentrum.b129.traffic.risc-software.at
-- route :  cur.bindermichl.b1.traffic.risc-software.at
-- route :  cur.franzosenhausweg.knotenlinz.traffic.risc-software.at
-- route :  cur.einwaerts.bindermichl.traffic.risc-software.at
-- route :  cur.urfahr.b127.traffic.risc-software.at
-- route :  cur.einwaerts.haselgraben.traffic.risc-software.at
-- route :  cur.florianerstrasse.knotenlinz.traffic.risc-software.at
-- route :  cur.auswaerts.haselgraben.traffic.risc-software.at
-- route :  cur.linz.autobahnbruecke.traffic.risc-software.at
-- route :  cur.linz.steyreggerbruecke.traffic.risc-software.at
-- route :  cur.zentrum.nibelungenbruecke.traffic.risc-software.at
-- route :  cur.rohrbach.b127.traffic.risc-software.at
-- route :  cur.urfahr.nibelungenbruecke.traffic.risc-software.at
-- route :  cur.wilhering.b129.traffic.risc-software.at
-- route :  cur.steyregg.steyreggerbruecke.traffic.risc-software.at
-- route :  cur.urfahr

From the retrived JSON we can select only the most usefull features for us :

```json
{u'attrs': {
      'average_speed'            : 39.6,
      'delay'                    : 8,
      'delayed_average_speed'    : 39.6,
      'total_distance'           : 4084,
      'total_duration'           : 359,
      'total_duration_in_traffic': 367
      }
}
```

For example the `total_duration` and `duration` are shown be on the same values.

In [4]:
import pandas as pd 
from pandas.io.json import json_normalize

To translate into a pandas dataframe, we will use a very usefull tool from pandas called json_normalize that create a data frame for us for free.

In [5]:
cur_df = {}
for c in cur_data:
    print '-- route : ', c
    cur_df[c] = json_normalize(cur_data[c])

-- route :  cur.wilhering.b129.traffic.risc-software.at
-- route :  cur.zentrum.nibelungenbruecke.traffic.risc-software.at
-- route :  cur.haid.knotenlinz.traffic.risc-software.at
-- route :  cur.bindermichl.b1.traffic.risc-software.at
-- route :  cur.urfahr.autobahnbruecke.traffic.risc-software.at
-- route :  cur.florianerstrasse.knotenlinz.traffic.risc-software.at
-- route :  cur.urfahr.nibelungenbruecke.traffic.risc-software.at
-- route :  cur.urfahr.b127.traffic.risc-software.at
-- route :  cur.rohrbach.b127.traffic.risc-software.at
-- route :  cur.franzosenhausweg.knotenlinz.traffic.risc-software.at
-- route :  cur.zentrum.b129.traffic.risc-software.at
-- route :  cur.linz.autobahnbruecke.traffic.risc-software.at
-- route :  cur.linz.steyreggerbruecke.traffic.risc-software.at
-- route :  cur.einwaerts.haselgraben.traffic.risc-software.at
-- route :  cur.einwaerts.bindermichl.traffic.risc-software.at
-- route :  cur.traun.b1.traffic.risc-software.at
-- route :  cur.auswaerts.haselg

After the normalisation of the json file, to make it easy to read, someone can use one root to extract the all the available features. But earlier we showed the most interetsing features so we are going to take those as the feature that we want to analyse

In [6]:
feat_traffic =  [
    'time',
    'data.attrs.delay'                    ,
    'data.attrs.average_speed'            ,
    'data.attrs.delayed_average_speed'    ,
    'data.attrs.total_distance'           ,
    'data.attrs.total_duration'           ,
    'data.attrs.total_duration_in_traffic'
]

As we can see we have different features names that have the radical "data.attrs" on it, and also we need to translate the linux time into date and hours. This will simplifiy the manipulation of the time series and help us understand the data. 

In [7]:
import numpy as np

df_routes = {}
for i,c in cur_df.items():
    _df_ = c
    _df_['time'] = pd.to_datetime(_df_['ts']/1000.0, unit='s')
    df_routes[i] = _df_[feat_traffic]

In [8]:
import time
from dateutil import parser, rrule
from datetime import datetime, time, date

def uniformise_dataframe(data = pd.DataFrame()):
    print "shape :: ", data.shape
    data_copy = data
    data_copy.is_copy = False
    data_copy['minute'] = data_copy.time.apply(lambda x: datetime(x.year, x.month, x.day, x.hour, x.minute) )
    data_copy = data_copy.reset_index().groupby(['minute']).mean()
    data_copy.reset_index(inplace=True)
    data_copy = data_copy.set_index(data_copy.minute.values)
    data_copy = data_copy.drop('minute', 1)
    data_copy = data_copy.drop('index' , 1)
    columns  = []
    for col in data_copy.columns:
        if 'data.attrs.' in col :
            data_copy[col.replace('data.attrs.','')] = data_copy[col] 
            columns.append(col.replace('data.attrs.',''))
        else:
            columns.append(col)
    return data_copy[columns]

In [9]:
for i in df_routes.keys():
    df_routes[i] = uniformise_dataframe(df_routes[i])

shape ::  (15391, 7)
shape ::  (15391, 7)
shape ::  (15391, 7)
shape ::  (15391, 7)
shape ::  (15392, 7)
shape ::  (15388, 7)
shape ::  (15390, 7)
shape ::  (15390, 7)
shape ::  (15392, 7)
shape ::  (15392, 7)
shape ::  (15392, 7)
shape ::  (15389, 7)
shape ::  (15392, 7)
shape ::  (15392, 7)
shape ::  (15391, 7)
shape ::  (15390, 7)
shape ::  (15390, 7)
shape ::  (15391, 7)
shape ::  (15391, 7)


Now let have a look on one root data frame and check that we are going in the right direction.

In [10]:
df_routes['cur.wilhering.b129.traffic.risc-software.at'].head()

,delay,average_speed,delayed_average_speed,total_distance,total_duration,total_duration_in_traffic
2016-12-21 13:30:00,0,39.6,43.2,4316,362,356
2016-12-21 13:40:00,0,39.6,43.2,4316,362,359
2016-12-21 13:50:00,0,39.6,39.6,4316,362,361
2016-12-21 13:51:00,0,39.6,43.2,4316,362,355
2016-12-21 14:01:00,0,39.6,43.2,4316,362,358


This looks perfect !

## Saving the data

Now let build some seasonal features such as :
* day of the year 
* hour in the day : we know that there is a reccurences during the day on the delay, for example when going for work
* day of the week : the delay should be different from weekend day then a working day

In [11]:
feat_time = ['day','dow','hod','working_day']

def fill_timefeatures(data = pd.DataFrame()):
    print "shape :: ", data.shape
    data_copy         = data
    data_copy.is_copy = False
    data_copy['day' ] = data.index.date
    data_copy['dow' ] = data.index.weekday   
    data_copy['hod' ] = data.index.hour
    # Is each time stamp on a working day (Mon-Fri)
    data_copy['working_day'] = (data_copy['dow'] >= 0) & (data_copy['dow'] <= 4)
    return data_copy

In [12]:
import os
if not os.path.exists('./data'):
    os.makedirs('./data')

In [13]:
for i in df_routes.keys():
    df_routes[i] = fill_timefeatures(df_routes[i])
    df_routes[i].to_csv( './data/' + i + '.csv')

shape ::  (15391, 6)
shape ::  (15391, 6)
shape ::  (15391, 6)
shape ::  (15391, 6)
shape ::  (15392, 6)
shape ::  (15388, 6)
shape ::  (15390, 6)
shape ::  (15390, 6)
shape ::  (15392, 6)
shape ::  (15392, 6)
shape ::  (15392, 6)
shape ::  (15389, 6)
shape ::  (15392, 6)
shape ::  (15392, 6)
shape ::  (15391, 6)
shape ::  (15390, 6)
shape ::  (15390, 6)
shape ::  (15391, 6)
shape ::  (15391, 6)


Now we have a set of data to work with and test different model and so on. Of course this method can be adpated for the real time model.